In [1]:
import numpy as np
import pandas as pd
import sys
import re
import lightgbm as lgb

In [2]:
data_path='F:\\study\\ml\\DataSet\\Drug_Molecule\\'
df_protein_train=pd.read_csv('%sdf_protein_train.csv' % (data_path))
df_protein_test=pd.read_csv('%sdf_protein_test.csv' % (data_path))
df_molecule=pd.read_csv('%sdf_molecule.csv'% (data_path))
df_affinity_train=pd.read_csv('%sdf_affinity_train.csv'% (data_path))
df_affinity_test=pd.read_csv('%sdf_affinity_test.csv'% (data_path))

In [3]:
df_affinity_test['Ki']=-11
data=pd.concat([df_affinity_train,df_affinity_test])

In [4]:
protein_concat = pd.concat([df_protein_train,df_protein_test],ignore_index=True)

In [5]:
df_protein_train.head()

,Protein_ID,Sequence
0,0,MDNVLPVDSDLSPNISTNTSEPNQFVQPAWQIVLWAAAYTVIVVTS...
1,1,MQWAVGRRWAWAALLLAVAAVLTQVVWLWLGTQSFVFQREEIAQLA...
2,2,MDPLNLSWYDDDLERQNWSRPFNGSEGKADRPHYNYYAMLLTLLIF...
3,3,MDSSTGPGNTSDCSDPLAQASCSPAPGSWLNLSHVDGNQSDPCGLN...
4,4,MEPVPSARAELQFSLLANVSDTFPSAFPSASANASGSPGARSASSL...


In [6]:
df_protein_test.head()

,Protein_ID,Sequence
0,6,MGRRRQGPAQPASELPARNACLLPNGSAWLPGWAEPDGNGSAGPQD...
1,8,MELNSSSRVDSEFRYTLFPIVYSIIFVLGIIANGYVLWVFARLYPS...
2,17,MSRARSHLRAALFLAAASARGITTQVAARRGLSAWPVPQEPSMEYQ...
3,25,MDAMKRGLCCVLLLCGAVFVSPSQEIHARFRRGARSYQVICRDEKT...
4,26,MAHVRGLQLPGCLALAALCSLVHSQHVFLAPQQARSLLQRVRRANT...


In [7]:
protein_concat.Sequence[0]

'MDNVLPVDSDLSPNISTNTSEPNQFVQPAWQIVLWAAAYTVIVVTSVVGNVVVMWIILAHKRMRTVTNYFLVNLAFAEASMAAFNTVVNFTYAVHNEWYYGLFYCKFHNFFPIAAVFASIYSMTAVAFDRYMAIIHPLQPRLSATATKVVICVIWVLALLLAFPQGYYSTTETMPSRVVCMIEWPEHPNKIYEKVYHICVTVLIYFLPLLVIGYAYTVVGITLWASEIPGDSSDRYHEQVSAKRKVVKMMIVVVCTFAICWLPFHIFFLLPYINPDLYLKKFIQQVYLAIMWLAMSSTMYNPIIYCCLNDRFRLGFKHAFRCCPFISAGDYEGLEMKSTRYLQTQGSVYKVSRLETTISTVVGAHEEEPEDGPKATPSSLDLTSNCSSRSDSKTMTESFSFSSNVLS'

In [8]:
data.head()

,Protein_ID,Molecule_ID,Ki
0,0,0,8.309804
1,1,1,10.292430
2,2,1,6.778586
3,1,2,10.769551
4,2,2,6.170568


In [9]:
protein_concat.head()

,Protein_ID,Sequence
0,0,MDNVLPVDSDLSPNISTNTSEPNQFVQPAWQIVLWAAAYTVIVVTS...
1,1,MQWAVGRRWAWAALLLAVAAVLTQVVWLWLGTQSFVFQREEIAQLA...
2,2,MDPLNLSWYDDDLERQNWSRPFNGSEGKADRPHYNYYAMLLTLLIF...
3,3,MDSSTGPGNTSDCSDPLAQASCSPAPGSWLNLSHVDGNQSDPCGLN...
4,4,MEPVPSARAELQFSLLANVSDTFPSAFPSASANASGSPGARSASSL...


In [10]:
df_molecule.head()

,Molecule_ID,Fingerprint,cyp_3a4,cyp_2c9,cyp_2d6,ames_toxicity,fathead_minnow_toxicity,tetrahymena_pyriformis_toxicity,honey_bee,cell_permeability,logP,renal_organic_cation_transporter,CLtotal,hia,biodegradation,Vdd,p_glycoprotein_inhibition,NOAEL,solubility,bbb
0,0,"1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0...",0.311594,0.742983,0.363031,0.623255,0.005905,NaN,0.800019,4.919215,NaN,0.479723,3.940952,0.029493,0.003006,24.211564,0.260743,-0.757354,-5.401525,0.087650
1,1,"1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",NaN,0.854373,0.280685,NaN,0.034951,2.174478,0.846931,4.723896,4.318473,0.492733,10.394106,NaN,0.016205,27.109723,0.511252,NaN,-5.089023,0.000608
2,2,"1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",0.102385,0.853763,0.197318,0.745154,0.030430,2.038387,0.854921,4.731113,NaN,0.403998,8.909931,0.011798,0.020116,20.560356,0.439826,-1.163596,-5.124717,0.000570
3,3,"1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",0.315907,0.855956,0.376436,NaN,0.033857,NaN,NaN,4.470623,NaN,0.699078,6.572888,0.010473,0.019070,14.791946,0.604665,-1.155990,-4.973178,0.029107
4,4,"1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",0.106763,0.844784,0.207776,NaN,NaN,1.895097,0.859958,4.723733,4.193617,0.438007,9.628845,0.011194,0.018409,22.261876,0.494917,-1.158525,-4.886913,0.000603


In [11]:
df_molecule.Fingerprint.head()

0    1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0...
1    1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...
2    1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...
3    1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...
4    1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...
Name: Fingerprint, dtype: object

In [12]:
feat=pd.DataFrame(df_molecule.Fingerprint.apply(lambda x:x.split(',')).tolist())
feat=feat.astype("int")

In [13]:
feat.head()

,0,1,2,3,4,5,6,7,8,9,...,157,158,159,160,161,162,163,164,165,166
0,1,0,0,1,0,1,0,1,0,0,...,1,0,0,0,0,1,0,0,1,1
1,1,1,0,1,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
2,1,1,0,1,0,1,0,0,0,0,...,1,0,0,0,0,1,0,0,0,1
3,1,1,0,1,0,1,0,0,0,0,...,1,0,0,0,0,1,0,0,0,1
4,1,1,0,1,0,1,0,0,0,0,...,1,0,0,0,0,1,0,0,0,1


In [14]:
feat.shape

(111216, 167)

In [15]:
feat.columns=['Fingerprint_{}'.format(i) for i in range(167)]

In [16]:
feat.head()

,Fingerprint_0,Fingerprint_1,Fingerprint_2,Fingerprint_3,Fingerprint_4,Fingerprint_5,Fingerprint_6,Fingerprint_7,Fingerprint_8,Fingerprint_9,...,Fingerprint_157,Fingerprint_158,Fingerprint_159,Fingerprint_160,Fingerprint_161,Fingerprint_162,Fingerprint_163,Fingerprint_164,Fingerprint_165,Fingerprint_166
0,1,0,0,1,0,1,0,1,0,0,...,1,0,0,0,0,1,0,0,1,1
1,1,1,0,1,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
2,1,1,0,1,0,1,0,0,0,0,...,1,0,0,0,0,1,0,0,0,1
3,1,1,0,1,0,1,0,0,0,0,...,1,0,0,0,0,1,0,0,0,1
4,1,1,0,1,0,1,0,0,0,0,...,1,0,0,0,0,1,0,0,0,1


In [17]:
data.head()

,Protein_ID,Molecule_ID,Ki
0,0,0,8.309804
1,1,1,10.292430
2,2,1,6.778586
3,1,2,10.769551
4,2,2,6.170568


In [18]:
feat['Molecule_ID']=df_molecule['Molecule_ID']

In [19]:
data=data.merge(feat,on='Molecule_ID',how='left')

In [20]:
df_protein_train.Sequence.head()

0    MDNVLPVDSDLSPNISTNTSEPNQFVQPAWQIVLWAAAYTVIVVTS...
1    MQWAVGRRWAWAALLLAVAAVLTQVVWLWLGTQSFVFQREEIAQLA...
2    MDPLNLSWYDDDLERQNWSRPFNGSEGKADRPHYNYYAMLLTLLIF...
3    MDSSTGPGNTSDCSDPLAQASCSPAPGSWLNLSHVDGNQSDPCGLN...
4    MEPVPSARAELQFSLLANVSDTFPSAFPSASANASGSPGARSASSL...
Name: Sequence, dtype: object

In [21]:
np.max(df_protein_train.Sequence.apply(lambda x:len(x)))

7073

In [22]:
np.min(df_protein_train.Sequence.apply(lambda x:len(x)))

26

In [23]:
df_protein_train['Sequence_len']=df_protein_train.Sequence.apply(lambda x:len(x))

In [24]:
texts=[[word for word in re.findall(r'.{3}',document)]
       for document in list(protein_concat.Sequence)]

In [25]:
texts[0]

['MDN',
 'VLP',
 'VDS',
 'DLS',
 'PNI',
 'STN',
 'TSE',
 'PNQ',
 'FVQ',
 'PAW',
 'QIV',
 'LWA',
 'AAY',
 'TVI',
 'VVT',
 'SVV',
 'GNV',
 'VVM',
 'WII',
 'LAH',
 'KRM',
 'RTV',
 'TNY',
 'FLV',
 'NLA',
 'FAE',
 'ASM',
 'AAF',
 'NTV',
 'VNF',
 'TYA',
 'VHN',
 'EWY',
 'YGL',
 'FYC',
 'KFH',
 'NFF',
 'PIA',
 'AVF',
 'ASI',
 'YSM',
 'TAV',
 'AFD',
 'RYM',
 'AII',
 'HPL',
 'QPR',
 'LSA',
 'TAT',
 'KVV',
 'ICV',
 'IWV',
 'LAL',
 'LLA',
 'FPQ',
 'GYY',
 'STT',
 'ETM',
 'PSR',
 'VVC',
 'MIE',
 'WPE',
 'HPN',
 'KIY',
 'EKV',
 'YHI',
 'CVT',
 'VLI',
 'YFL',
 'PLL',
 'VIG',
 'YAY',
 'TVV',
 'GIT',
 'LWA',
 'SEI',
 'PGD',
 'SSD',
 'RYH',
 'EQV',
 'SAK',
 'RKV',
 'VKM',
 'MIV',
 'VVC',
 'TFA',
 'ICW',
 'LPF',
 'HIF',
 'FLL',
 'PYI',
 'NPD',
 'LYL',
 'KKF',
 'IQQ',
 'VYL',
 'AIM',
 'WLA',
 'MSS',
 'TMY',
 'NPI',
 'IYC',
 'CLN',
 'DRF',
 'RLG',
 'FKH',
 'AFR',
 'CCP',
 'FIS',
 'AGD',
 'YEG',
 'LEM',
 'KST',
 'RYL',
 'QTQ',
 'GSV',
 'YKV',
 'SRL',
 'ETT',
 'IST',
 'VVG',
 'AHE',
 'EEP',
 'EDG',
 'PKA',


In [26]:
import gensim
from gensim.models import Word2Vec

D:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [27]:
n=256
model = Word2Vec(texts,size=n,window=8,min_count=1,negative=3,sg=1,sample=0.001,hs=1,workers=4)

In [28]:
#可见 词向量的行数,就是word的唯一值
model.wv.vectors.shape

(8803, 256)

In [29]:
model.wv.vectors

array([[-1.26869917e-01, -1.03526205e-01, -1.84263475e-02, ...,
         2.57575691e-01,  6.79191377e-04,  1.80304095e-01],
       [-1.14352323e-01,  1.59213506e-02,  3.29947546e-02, ...,
         3.70373987e-02,  2.76453979e-03,  2.95849770e-01],
       [ 6.87084571e-02,  9.42452461e-05,  9.55354944e-02, ...,
        -1.11521389e-02, -7.82401413e-02,  2.34136388e-01],
       ...,
       [ 1.13301817e-02,  2.11598594e-02,  3.79915163e-02, ...,
        -3.51575240e-02,  4.63222302e-02,  8.67318455e-03],
       [ 2.38356050e-02,  1.63490381e-02,  1.65521018e-02, ...,
        -2.47635059e-02,  2.47856565e-02,  4.24186625e-02],
       [ 8.42588115e-03,  1.91513579e-02,  2.76620518e-02, ...,
        -2.77590379e-02,  3.74362543e-02,  2.29933504e-02]], dtype=float32)

In [30]:
model['MDN']

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 0.13756281, -0.04676751, -0.05493842,  0.07089438, -0.03558761,
       -0.10467328,  0.11667224,  0.02547199, -0.07446018, -0.02003756,
       -0.14142634, -0.09344438,  0.12090958,  0.02349377, -0.06385283,
       -0.05628369,  0.09005675, -0.07535202,  0.05711317,  0.17146717,
        0.12490925, -0.15137938,  0.1313181 ,  0.07656562,  0.04189186,
        0.11802595,  0.17108501,  0.03940398, -0.19043311, -0.0321532 ,
       -0.08393037,  0.17257868,  0.03401726, -0.02185836, -0.00342051,
        0.05405201,  0.10824776,  0.13069025,  0.12237269, -0.18835779,
       -0.18052834,  0.12610511, -0.1909148 ,  0.01123147, -0.049833  ,
        0.10225788, -0.09062909,  0.07903695, -0.02517716, -0.05759479,
        0.082831  ,  0.14403832,  0.11868813,  0.00682444, -0.08247876,
       -0.10227367, -0.07818662,  0.00415118,  0.02872988, -0.13592564,
       -0.08895797, -0.30791813, -0.01006905,  0.05080298,  0.01213122,
       -0.028592  ,  0.08302479,  0.15938824,  0.01597791, -0.19

In [31]:
vectors=pd.DataFrame([model[word] for word in model.wv.vocab])
vectors['Word']=list(model.wv.vocab)
vectors.columns=['vec_{}'.format(i) for i in range(n)]+['Word']

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [32]:
wide_vec=pd.DataFrame()
result1=[]
aa=list(protein_concat.Protein_ID)
for i in range(len(texts)):
    result2=[]
    for w in range(len(texts[i])):
        result2.append(aa[i])
    result1.extend(result2)
wide_vec['Id']=result1

In [33]:
wide_vec.shape

(387090, 1)

In [34]:
type(wide_vec)

pandas.core.frame.DataFrame

In [35]:
wide_vec.head()

,Id
0,0
1,0
2,0
3,0
4,0


In [36]:
len(wide_vec.Id.unique())

2067

In [37]:
wide_vec2=wide_vec.copy()

In [38]:
result1=[]
for i in range(len(texts)):
    result2=[]
    for w in range(len(texts[i])):
        result2.append(texts[i][w])
    result1.extend(result2)
wide_vec['Word']=result1
    

In [39]:
wide_vec.head()

,Id,Word
0,0,MDN
1,0,VLP
2,0,VDS
3,0,DLS
4,0,PNI


In [40]:
#与词向量行数一致
len(wide_vec.Word.unique())

8803

In [41]:
del result1,result2

In [42]:
wide_vec=wide_vec.merge(vectors,on='Word',how='left')
wide_vec=wide_vec.drop('Word',axis=1)



In [43]:
wide_vec.head()

,Id,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,...,vec_246,vec_247,vec_248,vec_249,vec_250,vec_251,vec_252,vec_253,vec_254,vec_255
0,0,0.137563,-0.046768,-0.054938,0.070894,-0.035588,-0.104673,0.116672,0.025472,-0.074460,...,-0.091634,-0.084610,0.083121,0.018970,0.035215,0.090546,0.101058,0.033647,0.097215,-0.190379
1,0,-0.052099,-0.177900,0.022843,0.054791,-0.018395,-0.069148,0.314368,-0.075534,-0.107688,...,-0.139961,0.177871,-0.017168,0.025654,0.251029,0.124002,-0.061055,0.092871,-0.052091,0.171405
2,0,-0.081017,0.081302,-0.042634,-0.048618,0.017081,-0.021550,-0.002753,0.052030,-0.011437,...,-0.021542,-0.139972,0.080329,-0.203247,0.114790,-0.143233,0.087001,0.020377,-0.038373,0.053251
3,0,-0.031024,-0.017044,-0.196707,0.020375,-0.102956,-0.095594,0.316695,-0.005197,-0.022054,...,-0.068740,0.104368,-0.056292,-0.099376,0.180992,0.023315,-0.039458,-0.005400,0.089191,-0.000091
4,0,0.155931,0.106301,-0.012041,-0.100464,-0.253526,-0.043730,0.194764,-0.006726,0.125591,...,-0.017354,-0.074139,-0.104840,-0.137842,-0.027969,0.112089,-0.072234,-0.168985,0.327878,0.050328


In [44]:
wide_vec.rename(columns={'Id':'Protein_ID'},inplace=True)

In [45]:
wide_vec.head()

,Protein_ID,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,...,vec_246,vec_247,vec_248,vec_249,vec_250,vec_251,vec_252,vec_253,vec_254,vec_255
0,0,0.137563,-0.046768,-0.054938,0.070894,-0.035588,-0.104673,0.116672,0.025472,-0.074460,...,-0.091634,-0.084610,0.083121,0.018970,0.035215,0.090546,0.101058,0.033647,0.097215,-0.190379
1,0,-0.052099,-0.177900,0.022843,0.054791,-0.018395,-0.069148,0.314368,-0.075534,-0.107688,...,-0.139961,0.177871,-0.017168,0.025654,0.251029,0.124002,-0.061055,0.092871,-0.052091,0.171405
2,0,-0.081017,0.081302,-0.042634,-0.048618,0.017081,-0.021550,-0.002753,0.052030,-0.011437,...,-0.021542,-0.139972,0.080329,-0.203247,0.114790,-0.143233,0.087001,0.020377,-0.038373,0.053251
3,0,-0.031024,-0.017044,-0.196707,0.020375,-0.102956,-0.095594,0.316695,-0.005197,-0.022054,...,-0.068740,0.104368,-0.056292,-0.099376,0.180992,0.023315,-0.039458,-0.005400,0.089191,-0.000091
4,0,0.155931,0.106301,-0.012041,-0.100464,-0.253526,-0.043730,0.194764,-0.006726,0.125591,...,-0.017354,-0.074139,-0.104840,-0.137842,-0.027969,0.112089,-0.072234,-0.168985,0.327878,0.050328


In [46]:
del vectors

In [47]:
name=['vec_{}'.format(i) for i in range(0,n)]

In [48]:
feat=pd.DataFrame(wide_vec.groupby(['Protein_ID']).agg('mean')).reset_index()

In [49]:
feat.head()

,Protein_ID,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,...,vec_246,vec_247,vec_248,vec_249,vec_250,vec_251,vec_252,vec_253,vec_254,vec_255
0,0,0.022300,0.027626,0.023765,-0.064389,-0.048543,-0.041925,0.143631,0.028595,0.035610,...,-0.032484,0.055221,-0.042708,-0.070960,0.052889,0.102118,0.047818,-0.044043,0.080716,0.043507
1,1,0.039405,0.055791,0.027143,-0.055338,-0.013376,-0.048541,0.157726,0.048013,0.048243,...,-0.050729,-0.021534,-0.075454,-0.093803,0.069041,0.111112,0.000003,-0.046425,0.065472,0.059741
2,2,0.011542,0.039367,0.049039,-0.016768,0.001153,-0.087204,0.110648,0.074820,-0.013088,...,-0.064670,0.009709,-0.075063,-0.071671,0.035141,0.111390,-0.032266,-0.013980,0.093079,0.032202
3,3,0.021715,0.011077,-0.015415,-0.048907,0.011150,-0.050178,0.061540,0.090439,-0.006025,...,-0.015378,0.016878,-0.073407,-0.078351,0.083896,0.098823,0.028482,-0.054317,0.154408,0.090967
4,4,0.006263,0.073373,0.025799,-0.139532,-0.082339,-0.067253,0.177209,0.054213,0.014456,...,-0.043574,0.033309,-0.066560,-0.066380,0.115360,0.045837,0.087973,0.010657,0.068352,0.103001


In [50]:
feat.columns=['Protein_ID']+['mean_ci_{}'.format(i) for i in range(0,n)]
feat.head()

,Protein_ID,mean_ci_0,mean_ci_1,mean_ci_2,mean_ci_3,mean_ci_4,mean_ci_5,mean_ci_6,mean_ci_7,mean_ci_8,...,mean_ci_246,mean_ci_247,mean_ci_248,mean_ci_249,mean_ci_250,mean_ci_251,mean_ci_252,mean_ci_253,mean_ci_254,mean_ci_255
0,0,0.022300,0.027626,0.023765,-0.064389,-0.048543,-0.041925,0.143631,0.028595,0.035610,...,-0.032484,0.055221,-0.042708,-0.070960,0.052889,0.102118,0.047818,-0.044043,0.080716,0.043507
1,1,0.039405,0.055791,0.027143,-0.055338,-0.013376,-0.048541,0.157726,0.048013,0.048243,...,-0.050729,-0.021534,-0.075454,-0.093803,0.069041,0.111112,0.000003,-0.046425,0.065472,0.059741
2,2,0.011542,0.039367,0.049039,-0.016768,0.001153,-0.087204,0.110648,0.074820,-0.013088,...,-0.064670,0.009709,-0.075063,-0.071671,0.035141,0.111390,-0.032266,-0.013980,0.093079,0.032202
3,3,0.021715,0.011077,-0.015415,-0.048907,0.011150,-0.050178,0.061540,0.090439,-0.006025,...,-0.015378,0.016878,-0.073407,-0.078351,0.083896,0.098823,0.028482,-0.054317,0.154408,0.090967
4,4,0.006263,0.073373,0.025799,-0.139532,-0.082339,-0.067253,0.177209,0.054213,0.014456,...,-0.043574,0.033309,-0.066560,-0.066380,0.115360,0.045837,0.087973,0.010657,0.068352,0.103001


In [51]:
data.head()

,Protein_ID,Molecule_ID,Ki,Fingerprint_0,Fingerprint_1,Fingerprint_2,Fingerprint_3,Fingerprint_4,Fingerprint_5,Fingerprint_6,...,Fingerprint_157,Fingerprint_158,Fingerprint_159,Fingerprint_160,Fingerprint_161,Fingerprint_162,Fingerprint_163,Fingerprint_164,Fingerprint_165,Fingerprint_166
0,0,0,8.309804,1,0,0,1,0,1,0,...,1,0,0,0,0,1,0,0,1,1
1,1,1,10.292430,1,1,0,1,0,1,0,...,1,0,0,0,0,0,0,0,0,1
2,2,1,6.778586,1,1,0,1,0,1,0,...,1,0,0,0,0,0,0,0,0,1
3,1,2,10.769551,1,1,0,1,0,1,0,...,1,0,0,0,0,1,0,0,0,1
4,2,2,6.170568,1,1,0,1,0,1,0,...,1,0,0,0,0,1,0,0,0,1


In [52]:
feat.head()

,Protein_ID,mean_ci_0,mean_ci_1,mean_ci_2,mean_ci_3,mean_ci_4,mean_ci_5,mean_ci_6,mean_ci_7,mean_ci_8,...,mean_ci_246,mean_ci_247,mean_ci_248,mean_ci_249,mean_ci_250,mean_ci_251,mean_ci_252,mean_ci_253,mean_ci_254,mean_ci_255
0,0,0.022300,0.027626,0.023765,-0.064389,-0.048543,-0.041925,0.143631,0.028595,0.035610,...,-0.032484,0.055221,-0.042708,-0.070960,0.052889,0.102118,0.047818,-0.044043,0.080716,0.043507
1,1,0.039405,0.055791,0.027143,-0.055338,-0.013376,-0.048541,0.157726,0.048013,0.048243,...,-0.050729,-0.021534,-0.075454,-0.093803,0.069041,0.111112,0.000003,-0.046425,0.065472,0.059741
2,2,0.011542,0.039367,0.049039,-0.016768,0.001153,-0.087204,0.110648,0.074820,-0.013088,...,-0.064670,0.009709,-0.075063,-0.071671,0.035141,0.111390,-0.032266,-0.013980,0.093079,0.032202
3,3,0.021715,0.011077,-0.015415,-0.048907,0.011150,-0.050178,0.061540,0.090439,-0.006025,...,-0.015378,0.016878,-0.073407,-0.078351,0.083896,0.098823,0.028482,-0.054317,0.154408,0.090967
4,4,0.006263,0.073373,0.025799,-0.139532,-0.082339,-0.067253,0.177209,0.054213,0.014456,...,-0.043574,0.033309,-0.066560,-0.066380,0.115360,0.045837,0.087973,0.010657,0.068352,0.103001


In [53]:
data=data.merge(feat,on='Protein_ID',how='left')

In [54]:
data.head()

,Protein_ID,Molecule_ID,Ki,Fingerprint_0,Fingerprint_1,Fingerprint_2,Fingerprint_3,Fingerprint_4,Fingerprint_5,Fingerprint_6,...,mean_ci_246,mean_ci_247,mean_ci_248,mean_ci_249,mean_ci_250,mean_ci_251,mean_ci_252,mean_ci_253,mean_ci_254,mean_ci_255
0,0,0,8.309804,1,0,0,1,0,1,0,...,-0.032484,0.055221,-0.042708,-0.070960,0.052889,0.102118,0.047818,-0.044043,0.080716,0.043507
1,1,1,10.292430,1,1,0,1,0,1,0,...,-0.050729,-0.021534,-0.075454,-0.093803,0.069041,0.111112,0.000003,-0.046425,0.065472,0.059741
2,2,1,6.778586,1,1,0,1,0,1,0,...,-0.064670,0.009709,-0.075063,-0.071671,0.035141,0.111390,-0.032266,-0.013980,0.093079,0.032202
3,1,2,10.769551,1,1,0,1,0,1,0,...,-0.050729,-0.021534,-0.075454,-0.093803,0.069041,0.111112,0.000003,-0.046425,0.065472,0.059741
4,2,2,6.170568,1,1,0,1,0,1,0,...,-0.064670,0.009709,-0.075063,-0.071671,0.035141,0.111390,-0.032266,-0.013980,0.093079,0.032202


In [55]:
data.head()

,Protein_ID,Molecule_ID,Ki,Fingerprint_0,Fingerprint_1,Fingerprint_2,Fingerprint_3,Fingerprint_4,Fingerprint_5,Fingerprint_6,...,mean_ci_246,mean_ci_247,mean_ci_248,mean_ci_249,mean_ci_250,mean_ci_251,mean_ci_252,mean_ci_253,mean_ci_254,mean_ci_255
0,0,0,8.309804,1,0,0,1,0,1,0,...,-0.032484,0.055221,-0.042708,-0.070960,0.052889,0.102118,0.047818,-0.044043,0.080716,0.043507
1,1,1,10.292430,1,1,0,1,0,1,0,...,-0.050729,-0.021534,-0.075454,-0.093803,0.069041,0.111112,0.000003,-0.046425,0.065472,0.059741
2,2,1,6.778586,1,1,0,1,0,1,0,...,-0.064670,0.009709,-0.075063,-0.071671,0.035141,0.111390,-0.032266,-0.013980,0.093079,0.032202
3,1,2,10.769551,1,1,0,1,0,1,0,...,-0.050729,-0.021534,-0.075454,-0.093803,0.069041,0.111112,0.000003,-0.046425,0.065472,0.059741
4,2,2,6.170568,1,1,0,1,0,1,0,...,-0.064670,0.009709,-0.075063,-0.071671,0.035141,0.111390,-0.032266,-0.013980,0.093079,0.032202


In [56]:
import lightgbm as lgb

In [57]:
train_feat=data[data['Ki'] > -11].fillna(0)
testt_feat=data[data['Ki']<=-11].fillna(0)

In [58]:
testt_feat.head()

,Protein_ID,Molecule_ID,Ki,Fingerprint_0,Fingerprint_1,Fingerprint_2,Fingerprint_3,Fingerprint_4,Fingerprint_5,Fingerprint_6,...,mean_ci_246,mean_ci_247,mean_ci_248,mean_ci_249,mean_ci_250,mean_ci_251,mean_ci_252,mean_ci_253,mean_ci_254,mean_ci_255
165084,6,37,-11.0,1,1,0,1,0,1,0,...,-0.028833,0.029607,-0.084008,-0.09174,0.035888,0.103606,0.056508,-0.049278,0.127997,0.083206
165085,6,36,-11.0,1,1,0,1,0,1,0,...,-0.028833,0.029607,-0.084008,-0.09174,0.035888,0.103606,0.056508,-0.049278,0.127997,0.083206
165086,6,38,-11.0,1,1,0,1,0,1,0,...,-0.028833,0.029607,-0.084008,-0.09174,0.035888,0.103606,0.056508,-0.049278,0.127997,0.083206
165087,6,39,-11.0,1,1,0,1,0,1,0,...,-0.028833,0.029607,-0.084008,-0.09174,0.035888,0.103606,0.056508,-0.049278,0.127997,0.083206
165088,6,228,-11.0,1,1,1,1,0,1,1,...,-0.028833,0.029607,-0.084008,-0.09174,0.035888,0.103606,0.056508,-0.049278,0.127997,0.083206


In [59]:
label_x=train_feat['Ki']
label_y=testt_feat['Ki']


In [60]:
train_feat.iloc[-5:]

,Protein_ID,Molecule_ID,Ki,Fingerprint_0,Fingerprint_1,Fingerprint_2,Fingerprint_3,Fingerprint_4,Fingerprint_5,Fingerprint_6,...,mean_ci_246,mean_ci_247,mean_ci_248,mean_ci_249,mean_ci_250,mean_ci_251,mean_ci_252,mean_ci_253,mean_ci_254,mean_ci_255
165079,117,111211,9.259637,1,0,0,1,0,0,1,...,-0.016492,0.043515,-0.030644,-0.069266,0.063256,0.067432,-0.018508,-0.036131,0.078214,0.083196
165080,117,111212,9.721247,1,0,0,1,0,0,1,...,-0.016492,0.043515,-0.030644,-0.069266,0.063256,0.067432,-0.018508,-0.036131,0.078214,0.083196
165081,117,111213,9.508638,1,0,0,1,0,0,1,...,-0.016492,0.043515,-0.030644,-0.069266,0.063256,0.067432,-0.018508,-0.036131,0.078214,0.083196
165082,117,111214,10.040958,1,0,0,1,0,0,1,...,-0.016492,0.043515,-0.030644,-0.069266,0.063256,0.067432,-0.018508,-0.036131,0.078214,0.083196
165083,117,111215,9.744727,1,0,0,1,0,0,1,...,-0.016492,0.043515,-0.030644,-0.069266,0.063256,0.067432,-0.018508,-0.036131,0.078214,0.083196


In [61]:
testt_feat.head()

,Protein_ID,Molecule_ID,Ki,Fingerprint_0,Fingerprint_1,Fingerprint_2,Fingerprint_3,Fingerprint_4,Fingerprint_5,Fingerprint_6,...,mean_ci_246,mean_ci_247,mean_ci_248,mean_ci_249,mean_ci_250,mean_ci_251,mean_ci_252,mean_ci_253,mean_ci_254,mean_ci_255
165084,6,37,-11.0,1,1,0,1,0,1,0,...,-0.028833,0.029607,-0.084008,-0.09174,0.035888,0.103606,0.056508,-0.049278,0.127997,0.083206
165085,6,36,-11.0,1,1,0,1,0,1,0,...,-0.028833,0.029607,-0.084008,-0.09174,0.035888,0.103606,0.056508,-0.049278,0.127997,0.083206
165086,6,38,-11.0,1,1,0,1,0,1,0,...,-0.028833,0.029607,-0.084008,-0.09174,0.035888,0.103606,0.056508,-0.049278,0.127997,0.083206
165087,6,39,-11.0,1,1,0,1,0,1,0,...,-0.028833,0.029607,-0.084008,-0.09174,0.035888,0.103606,0.056508,-0.049278,0.127997,0.083206
165088,6,228,-11.0,1,1,1,1,0,1,1,...,-0.028833,0.029607,-0.084008,-0.09174,0.035888,0.103606,0.056508,-0.049278,0.127997,0.083206


In [62]:
label_x.head()

0     8.309804
1    10.292430
2     6.778586
3    10.769551
4     6.170568
Name: Ki, dtype: float64

In [63]:
train_feat = train_feat.drop('Ki',axis=1)
testt_feat = testt_feat.drop('Ki',axis=1)
train_feat = train_feat.drop('Protein_ID',axis=1)
testt_feat = testt_feat.drop('Protein_ID',axis=1)
train_feat = train_feat.drop('Molecule_ID',axis=1)
testt_feat = testt_feat.drop('Molecule_ID',axis=1)

In [64]:
train=lgb.Dataset(train_feat,label=label_x)
test=lgb.Dataset(testt_feat,label=label_y,reference=train)

In [65]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression_l2',
    'metric': 'l2',
    #'objective': 'multiclass',
    #'metric': 'multi_error',
    #'num_class':5,
    'min_child_weight': 3,
    'num_leaves': 2 ** 5,
    'lambda_l2': 10,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'colsample_bylevel': 0.7,
    'learning_rate': 0.05,
    'tree_method': 'exact',
    'seed': 2017,
    'nthread': 4,
    'silent': True
    }

In [75]:
import time

In [76]:
%%time
num_round = 3000
gbm = lgb.train(params, 
                  train, 
                  num_round, 
                  verbose_eval=50,
                  valid_sets=[train,test]
                  )

[50]	training's l2: 1.71974	valid_1's l2: 341.931
[100]	training's l2: 1.5068	valid_1's l2: 340.056
[150]	training's l2: 1.39401	valid_1's l2: 339.32
[200]	training's l2: 1.3184	valid_1's l2: 338.397
[250]	training's l2: 1.25952	valid_1's l2: 338.179
[300]	training's l2: 1.20941	valid_1's l2: 337.833
[350]	training's l2: 1.16793	valid_1's l2: 337.391
[400]	training's l2: 1.1339	valid_1's l2: 336.946
[450]	training's l2: 1.09916	valid_1's l2: 336.845
[500]	training's l2: 1.06798	valid_1's l2: 336.799
[550]	training's l2: 1.04286	valid_1's l2: 336.786
[600]	training's l2: 1.01604	valid_1's l2: 336.675
[650]	training's l2: 0.992918	valid_1's l2: 336.436
[700]	training's l2: 0.971436	valid_1's l2: 336.275
[750]	training's l2: 0.953452	valid_1's l2: 336.08
[800]	training's l2: 0.937042	valid_1's l2: 336.071
[850]	training's l2: 0.919532	valid_1's l2: 335.947
[900]	training's l2: 0.903254	valid_1's l2: 335.76
[950]	training's l2: 0.888215	valid_1's l2: 335.708
[1000]	training's l2: 0.873661	

In [67]:
from sklearn.metrics import mean_squared_error

In [68]:
preds_train= gbm.predict(train_feat)

In [69]:
mse=mean_squared_error(label_x,preds_train)
np.sqrt(mse)

0.7673825445493138

In [70]:
train_feat.head()

,Fingerprint_0,Fingerprint_1,Fingerprint_2,Fingerprint_3,Fingerprint_4,Fingerprint_5,Fingerprint_6,Fingerprint_7,Fingerprint_8,Fingerprint_9,...,mean_ci_246,mean_ci_247,mean_ci_248,mean_ci_249,mean_ci_250,mean_ci_251,mean_ci_252,mean_ci_253,mean_ci_254,mean_ci_255
0,1,0,0,1,0,1,0,1,0,0,...,-0.032484,0.055221,-0.042708,-0.070960,0.052889,0.102118,0.047818,-0.044043,0.080716,0.043507
1,1,1,0,1,0,1,0,0,0,0,...,-0.050729,-0.021534,-0.075454,-0.093803,0.069041,0.111112,0.000003,-0.046425,0.065472,0.059741
2,1,1,0,1,0,1,0,0,0,0,...,-0.064670,0.009709,-0.075063,-0.071671,0.035141,0.111390,-0.032266,-0.013980,0.093079,0.032202
3,1,1,0,1,0,1,0,0,0,0,...,-0.050729,-0.021534,-0.075454,-0.093803,0.069041,0.111112,0.000003,-0.046425,0.065472,0.059741
4,1,1,0,1,0,1,0,0,0,0,...,-0.064670,0.009709,-0.075063,-0.071671,0.035141,0.111390,-0.032266,-0.013980,0.093079,0.032202


In [71]:
from sklearn.cross_validation  import cross_val_score,train_test_split

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
